# Power electricity power consumption prediction model

Let's first start by importing the needed libraries

In [1]:
from pyspark.sql import SparkSession

And create a local parallel spark session

In [2]:
spark = SparkSession.builder \
        .master("local[*]") \
        .appName("Power electricity prediction") \
        .getOrCreate()

The we read the dataframe we need to apply the prediction on

In [3]:
train = spark.read.csv('../data/engineered/powerelectricity_train.csv', header=True, inferSchema=True)
test = spark.read.csv('../data/engineered/powerelectricity_test.csv', header=True, inferSchema=True)

Now we will test some regression models and evaluate them using rmse

In [4]:
from pyspark.ml import Pipeline
from pyspark.ml.regression import RandomForestRegressor, GBTRegressor, LinearRegression
from pyspark.ml.feature import VectorIndexer
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.feature import VectorAssembler

In [5]:
train_ep = train.drop('rms_current')
train_ep = train_ep.drop('electric_power')

#train_rms = train.drop('electric_power')

test_ep = test.drop('rms_current')
test_ep = test_ep.drop('electric_power')

#test_rms = test.drop('electric_power')

features_ep = train_ep.columns.copy()
#features_ep.remove('electric_power')
features_ep.remove('delta_y')

#features_rms = train_rms.columns.copy()
#features_rms.remove('rms_current')

In [6]:
assembler_pe_train = VectorAssembler(inputCols=features_ep,outputCol="features")
train_ep = assembler_pe_train.transform(train_ep)

Let's start by predicting electric_power

In [7]:
rf_pe = RandomForestRegressor(featuresCol="features",labelCol='delta_y',seed=42)

In [8]:
%%time

model_pe = rf_pe.fit(train_ep)

CPU times: user 26.3 ms, sys: 39.5 ms, total: 65.8 ms
Wall time: 3min 38s


In [9]:
assembler_pe_test = VectorAssembler(inputCols=features_ep,outputCol="features")
test_ep = assembler_pe_test.transform(test_ep)

In [10]:
predictions = model_pe.transform(test_ep)

In [11]:
evaluator_rmse = RegressionEvaluator(
    labelCol="delta_y", predictionCol="prediction", metricName="rmse")
rmse = evaluator_rmse.evaluate(predictions)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

evaluator_r2 = RegressionEvaluator(
    labelCol="delta_y", predictionCol="prediction", metricName="r2")
r2 = evaluator_r2.evaluate(predictions)
print("R squared (r2) on test data = %g" % r2)

evaluator_mae = RegressionEvaluator(
    labelCol="delta_y", predictionCol="prediction", metricName="mae")
mae = evaluator_mae.evaluate(predictions)
print("Mean Average Error (RMSE) on test data = %g" % mae)

Root Mean Squared Error (RMSE) on test data = 0.125874
R squared (r2) on test data = 0.370969
Mean Average Error (RMSE) on test data = 0.104562


In [12]:
predictions.select(["delta_y", "prediction"]).show(100000)

+--------------------+--------------------+
|             delta_y|          prediction|
+--------------------+--------------------+
|0.004969091889392985| 0.12325618509972529|
| 0.48309792252243433| 0.14668820473174274|
| 0.48309792252243433|  0.1579532878835346|
| 0.48309792252243433| 0.15850629994706583|
| 0.47091821780013765| 0.11118005713938592|
|  0.4757933520154678| 0.11213912987348629|
|  0.4669144258099456| 0.11550936961013095|
| 0.47593573864292704| 0.09384441177439144|
|  0.4826807225147509| 0.12542139005621145|
| 0.47259031195294954| 0.10578640816105586|
|  0.4555965524388075| 0.13851408795436598|
|  0.4703448213914534| 0.12789796931598385|
|  0.3225400748647185| 0.07007680017607733|
|  0.3225400748647185|  0.0988579501426729|
|  0.3225400748647185| 0.07859997823039352|
|  0.3225400748647185| 0.06765753806964872|
|  0.3225400748647185| 0.07483209656410084|
|  0.3225400748647185|0.037651812178469274|
|  0.3225400748647185| 0.07946697575892561|
|  0.3225400748647185| 0.0834674

And finally save to a file the predictions

In [13]:
import os

try:
    os.mkdir('../predictions/')
except:
    pass

preds = predictions.select(["prediction"])
preds.toPandas().to_csv('../predictions/preds.csv')